In [ ]:
from solve import solve
import os, sys
import numpy as np
from InterpFromMeshToMesh2d import InterpFromMeshToMesh2d
from shapely import LineString
from shapely.geometry import Point, Polygon

###############################
###############################
### make issm solve quietly ###
###############################
###############################

class Null:
    def write(self, *_): pass
    def flush(self): pass

null = Null()

def solve_quiet(md, kind):
    dn = os.open(os.devnull, os.O_WRONLY)
    o1, o2 = os.dup(1), os.dup(2)
    s1, s2 = sys.stdout, sys.stderr
    sys.stdout = null; sys.stderr = null
    os.dup2(dn, 1); os.dup2(dn, 2); os.close(dn)
    try:
        return solve(md, kind)
    finally:
        os.dup2(o1, 1); os.dup2(o2, 2)
        os.close(o1); os.close(o2)
        sys.stdout = s1; sys.stderr = s2

#########################
#########################
### diagnostic solver ###
#########################
#########################

def diagnostic_solve(md, **kwargs):
    
    T = kwargs['temperature']
    C = kwargs['friction']
    p = kwargs.get('friction_p', 1)
    q = kwargs.get('friction_q', 1)
    H = kwargs['thickness']
    u = kwargs['velocity']
    Γ_x, Γ_y = kwargs['dirichlet_x'], kwargs['dirichlet_y']
    name = kwargs.get('name', 'name')
    quiet = kwargs.get('quiet', True)

    num_e = md.mesh.numberofelements
    num_v = md.mesh.numberofvertices

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    ################
    ### rheology ###
    ################
    
    T_t = 263.15 #transition temperature (K)
    Q = 6e4 if T < T_t else 115e3 #activation enegy (J/mol), depends on temperature
    A_0 = 3.5e-25 #rate prefactor (s^-1 Pa^-3)
    R = 8.314 #gas constant (J mol^-1 K^-1)
    A_val = A_0*np.exp(-Q/R*(1/T - 1/T_t)) #final rate factor (s^-1 Pa^-3)
    B_val = A_val**(-1/md.materials.rheology_n)
    md.materials.rheology_B = B_val*np.ones(num_v)

    ################
    ### friction ###
    ################

    md.friction.coefficient = C*np.ones(num_e) if np.isscalar(C) else C
    md.friction.p = p*np.ones(num_e) if np.isscalar(p) else p
    md.friction.q = q*np.ones(num_e) if np.isscalar(q) else q

    ################
    ### geometry ###
    ################    

    md.geometry.thickness = H
    md.geometry.surface = (1 - ϱ)*H
    md.geometry.base = md.geometry.surface - md.geometry.thickness

    ###########################
    ### boundary conditions ###
    ###########################

    md.stressbalance.spcvx = np.nan*np.ones(num_v) #initialize single-point constraints (SPCs) on vx and vy
    md.stressbalance.spcvy = np.nan*np.ones(num_v) #wherever these remain nans, the model assumes an ice front
    md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
    md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]

    ###########################
    ### additional settings ###
    ###########################

    md.stressbalance.referential = np.nan*np.ones((num_v, 6))
    md.stressbalance.loadingforce = np.zeros((num_v, 3))
    md.miscellaneous.name = name

    #############
    ### solve ###
    ############# 

    md = solve_quiet(md, 'StressBalance') if quiet else solve(md, 'StressBalance')
    u_x, u_y = md.results.StressbalanceSolution.Vx.flatten(), md.results.StressbalanceSolution.Vy.flatten()
    return (u_x, u_y)

#########################
#########################
### prognostic solver ###
#########################
#########################

def prognostic_solve(md, Δt, **kwargs):
    
    u = kwargs['velocity']
    H = kwargs['thickness']
    H_in = kwargs['thickness_inflow']
    Γ_in = kwargs['inflow_boundary']
    a_dot = kwargs.get('accumulation', 0)
    quiet = kwargs.get('quiet', True)
    moving_front = kwargs.get('moving_front', False)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    num_v = md.mesh.numberofvertices
    x, y = md.mesh.x, md.mesh.y

    ###################
    ### solver type ###
    ###################

    md.transient.isstressbalance = 1 #solve the stress balance: no
    md.transient.ismasstransport = 1 #solve mass transport: yes
    md.transient.isthermal = 0 #solve heat transport: no

    ###############################
    ### timestepping parameters ###
    ###############################

    md.timestepping.time_step = Δt
    md.timestepping.final_time = Δt

    #####################################
    ### thickness boundary conditions ###
    #####################################

    md.geometry.thickness = H
    md.geometry.surface = (1 - ϱ)*H
    md.masstransport.spcthickness = np.nan*np.ones(num_v) #initialize thickness boundary conditions as nans
    md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]


    ########################
    ### other parameters ###
    ########################
    
    md.smb.initialize(md)
    md.smb.mass_balance = a_dot*np.ones(num_v) if np.isscalar(a_dot) else a_dot
    md.basalforcings.initialize(md)
    md.basalforcings.groundedice_melting_rate = np.zeros(num_v)
    md.basalforcings.floatingice_melting_rate = np.zeros(num_v)
    md.initialization.vx = u[0]
    md.initialization.vy = u[1] 

    #############
    ### solve ###
    ############# 

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    H = solution.Thickness.flatten()
    s = (1 - ϱ)*H
    md.geometry.thickness = H
    md.geometry.surface = s


    ##########################
    ### terminus evolution ###
    ##########################
    
    if moving_front: 

        H_min = kwargs['thickness_infill']
        terminus_curve = md.miscellaneous.terminus
        xs, ys = map(np.asarray, terminus_curve.xy)
        x_max, x_min, y_max, y_min = x.max(), x.min(), y.max(), y.min()
    
        x_shift = InterpFromMeshToMesh2d(md.mesh.elements, x, y, u[0]*Δt, xs, ys).ravel()
        y_shift = InterpFromMeshToMesh2d(md.mesh.elements, x, y, u[1]*Δt, xs, ys).ravel()
        
        xs_new = xs + x_shift
        ys_new = ys + y_shift
        terminus_curve = LineString(np.column_stack((xs_new, ys_new)))
        md.miscellaneous.terminus = terminus_curve
        
        Ω_ice = Polygon([
            (x_min, y_min),
            (x_min, y_max),
            *zip(xs_new[::-1], ys_new[::-1]),
            (x_min, y_min),
        ])
        
        dist = np.array([terminus_curve.distance(Point(xi, yi)) for xi, yi in zip(x, y)])
        inside = np.array([Ω_ice.covers(Point(xi, yi)) for xi, yi in zip(x, y)])
        
        md.mask.ice_levelset = np.where(inside, -dist, dist)
        
        H[inside] = np.maximum(H[inside], H_min)
        s[inside] = (1 - ϱ)*H[inside]
        md.geometry.thickness = H
        md.geometry.surface = s
        
    return H

############################################
############################################
### coupled diagnostic-prognostic solver ###
############################################
############################################

def coupled_solve(md, Δt, **kwargs):
    
    T = kwargs['temperature']
    C = kwargs['friction']
    p = kwargs.get('friction_p', 1)
    q = kwargs.get('friction_q', 1)
    H = kwargs['thickness']
    u = kwargs['velocity']
    Γ_x, Γ_y = kwargs['dirichlet_x'], kwargs['dirichlet_y']
    name = kwargs.get('name', 'name')
    quiet = kwargs.get('quiet', True)
    H_in = kwargs['thickness_inflow']
    Γ_in = kwargs['inflow_boundary']
    a_dot = kwargs.get('accumulation', 0)
    moving_front = kwargs.get('moving_front', False)

    num_e = md.mesh.numberofelements
    num_v = md.mesh.numberofvertices
    x, y = md.mesh.x, md.mesh.y

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    ###################
    ### solver type ###
    ###################

    md.transient.isstressbalance = 1 #solve the stress balance: yes
    md.transient.ismasstransport = 1 #solve mass transport: yes
    md.transient.isthermal = 0 #solve heat transport: no

    ###############################
    ### timestepping parameters ###
    ###############################

    md.timestepping.time_step = Δt
    md.timestepping.final_time = Δt

    ################
    ### rheology ###
    ################
    
    T_t = 263.15 #transition temperature (K)
    Q = 6e4 if T < T_t else 115e3 #activation enegy (J/mol), depends on temperature
    A_0 = 3.5e-25 #rate prefactor (s^-1 Pa^-3)
    R = 8.314 #gas constant (J mol^-1 K^-1)
    A_val = A_0*np.exp(-Q/R*(1/T - 1/T_t)) #final rate factor (s^-1 Pa^-3)
    B_val = A_val**(-1/md.materials.rheology_n)
    md.materials.rheology_B = B_val*np.ones(num_v)

    ################
    ### friction ###
    ################

    md.friction.coefficient = C*np.ones(num_e) if np.isscalar(C) else C
    md.friction.p = p*np.ones(num_e) if np.isscalar(p) else p
    md.friction.q = q*np.ones(num_e) if np.isscalar(q) else q

    ################
    ### geometry ###
    ################    

    md.geometry.thickness = H
    md.geometry.surface = (1 - ϱ)*H
    md.geometry.base = md.geometry.surface - md.geometry.thickness

    ###########################
    ### boundary conditions ###
    ###########################

    md.stressbalance.spcvx = np.nan*np.ones(num_v) #initialize single-point constraints (SPCs) on vx and vy
    md.stressbalance.spcvy = np.nan*np.ones(num_v) #wherever these remain nans, the model assumes an ice front
    md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
    md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]
    md.masstransport.spcthickness = np.nan*np.ones(num_v) #initialize thickness boundary conditions as nans
    md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]

    ###########################
    ### additional settings ###
    ###########################

    md.stressbalance.referential = np.nan*np.ones((num_v, 6))
    md.stressbalance.loadingforce = np.zeros((num_v, 3))
    md.miscellaneous.name = name
    md.smb.initialize(md)
    md.smb.mass_balance = a_dot*np.ones(num_v) if np.isscalar(a_dot) else a_dot
    md.basalforcings.initialize(md)
    md.basalforcings.groundedice_melting_rate = np.zeros(num_v)
    md.basalforcings.floatingice_melting_rate = np.zeros(num_v)
    md.initialization.vx = u[0]
    md.initialization.vy = u[1] 

    #############
    ### solve ###
    #############

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    u_x, u_y = solution.Vx.flatten(), solution.Vy.flatten()
    H = solution.Thickness.flatten()
    s = (1 - ϱ)*H
    md.geometry.thickness = H
    md.geometry.surface = s

    ##########################
    ### terminus evolution ###
    ##########################
    
    if moving_front: 

        H_min = kwargs['thickness_infill']
        terminus_curve = md.miscellaneous.terminus
        xs, ys = map(np.asarray, terminus_curve.xy)
        x_max, x_min, y_max, y_min = x.max(), x.min(), y.max(), y.min()
    
        x_shift = InterpFromMeshToMesh2d(md.mesh.elements, x, y, u_x*Δt, xs, ys).ravel()
        y_shift = InterpFromMeshToMesh2d(md.mesh.elements, x, y, u_y*Δt, xs, ys).ravel()
        
        xs_new = xs + x_shift
        ys_new = ys + y_shift
        terminus_curve = LineString(np.column_stack((xs_new, ys_new)))
        md.miscellaneous.terminus = terminus_curve
        
        Ω_ice = Polygon([
            (x_min, y_min),
            (x_min, y_max),
            *zip(xs_new[::-1], ys_new[::-1]),
            (x_min, y_min),
        ])
        
        dist = np.array([terminus_curve.distance(Point(xi, yi)) for xi, yi in zip(x, y)])
        inside = np.array([Ω_ice.covers(Point(xi, yi)) for xi, yi in zip(x, y)])
        
        md.mask.ice_levelset = np.where(inside, -dist, dist)
        
        H[inside] = np.maximum(H[inside], H_min)
        s[inside] = (1 - ϱ)*H[inside]
        md.geometry.thickness = H
        md.geometry.surface = s

    return (u_x, u_y), H

In [ ]:
# !jupyter nbconvert --to script make_issm_like_icepack.ipynb